# Simulación del tráfico en un carril.

Primero se simulará para dos carros. La ecuación que simulará la velocidad de los vehículos es el mínimo entre

$$\dot{x_i} = \frac{1}{T_i}(x_{i-1}-x_i -\frac{l_i + l_{i-1}}{2}) 
$$ 

y

$$ \dot{x_i} = v_{max_{i}} $$

In [1]:
#Llamando al paquete de Taylor

using TaylorSeries

 in depwarn at deprecated.jl:73
 in call at deprecated.jl:50
 [inlined code] from /home/windie/.julia/v0.4/TaylorSeries/src/Taylor1.jl:127
 in anonymous at no file:0
 in include_string at loading.jl:266
 in execute_request_0x535c5df2 at /home/windie/.julia/v0.4/IJulia/src/execute_request.jl:177
 in eventloop at /home/windie/.julia/v0.4/IJulia/src/IJulia.jl:141
 in anonymous at task.jl:447
while loading /home/windie/.julia/v0.4/TaylorSeries/src/Taylor1.jl, in expression starting on line 143
 in depwarn at deprecated.jl:73
 in call at deprecated.jl:50
 [inlined code] from /home/windie/.julia/v0.4/TaylorSeries/src/Taylor1.jl:127
 in anonymous at no file:0
 in include_string at loading.jl:266
 in execute_request_0x535c5df2 at /home/windie/.julia/v0.4/IJulia/src/execute_request.jl:177
 in eventloop at /home/windie/.julia/v0.4/IJulia/src/IJulia.jl:141
 in anonymous at task.jl:447
while loading /home/windie/.julia/v0.4/TaylorSeries/src/Taylor1.jl, in expression starting on line 152
 in depwar

In [2]:
#Definiendo parámetros. Sólo para fijar ideas.

x1_0 = 0.0;
x2_0 = pi;
l1 = pi/16;
l2 = l1;
V1 = pi/4;
V2 = pi/2;

per1=1;
per2=1;



A continuación se escribe la función `modulo2pi` que encuentra la diferencia entre dos ángulos en módulo 2$\pi$

In [3]:
function modulo2pi(α,β)
    #Esta función calcula la diferencia entre dos ángulos y la devuelve 
    #módulo 2π.
    δ=α-β 
    
    if δ<0
        δ = 2π + δ
    else 
        δ = δ 
    end
    
    return δ
end

 in depwarn at deprecated.jl:73
 in call at deprecated.jl:50
 in include_string at loading.jl:266
 in execute_request_0x535c5df2 at /home/windie/.julia/v0.4/IJulia/src/execute_request.jl:177
 in eventloop at /home/windie/.julia/v0.4/IJulia/src/IJulia.jl:141
 in anonymous at task.jl:447
while loading /home/windie/.julia/v0.4/TaylorSeries/src/printing.jl, in expression starting on line 149
 in depwarn at deprecated.jl:73
 in call at deprecated.jl:50
 in include_string at loading.jl:266
 in execute_request_0x535c5df2 at /home/windie/.julia/v0.4/IJulia/src/execute_request.jl:177
 in eventloop at /home/windie/.julia/v0.4/IJulia/src/IJulia.jl:141
 in anonymous at task.jl:447
while loading /home/windie/.julia/v0.4/TaylorSeries/src/printing.jl, in expression starting on line 154


modulo2pi (generic function with 1 method)

In [4]:
#La siguiente función calcula los coeficientes de la velocidad de seguridad

function velocidad_sec{R<:Real }(T::R, car::Taylor1{R}, frontcar::Taylor1{R} , lfcar::R)
    k = max(car.order, frontcar.order)
    
    vel = taylor1_variable(Float64, k)
     
    vel.coeffs[1] = 1/T * modulo2pi(frontcar.coeffs[1]-lfcar,car.coeffs[1])
    
    for i = 2 : k
        vel.coeffs[i] = 1/T * (frontcar.coeffs[i]-car.coeffs[i])
    end
        
    
    return vel
end



velocidad_sec (generic function with 1 method)

Ahora se definen un par de variables tipo `Taylor1` que tendrá información de las series de taylor de las funciones $x_1 (t)$ y $x_2 (t)$.

Ahora, a encontrar los siguientes coeficientes. Usando que $$ x_{n+1} = f_n /(n+1) $$

Es momento de avanzar. definamos la épsilon y la función para calcular el siguiente paso.

Sabemos que el paso siguiente está dado por $ h = \min[\, (\epsilon/x^{[p-1]})^{1/(p-1)}, (\epsilon/x^{[p]})^{1/p}\, ], $

In [11]:
function stepsize{T<:Real}(x::Taylor1{T}, epsilon::Float64)
    ord = x.order
    h = Inf
    
    if ord > 1
        
        for k in [ord-1, ord]
            kinv = 1.0/k
            aux = abs( x.coeffs[k+1] )
            h = min(h, (epsilon/aux)^kinv)
        end
    
    elseif ord == 1
        
        
        
        aux = abs( x.coeffs[2])
        
        h = min(h, (epsilon/aux))
        
    end
        
    
    
    return h
end

stepsize (generic function with 1 method)

Necesitamos una función que sólo con las condiciones iniciales (y otros parámetros) , calcule las series de dos coches.

In [6]:
function series(x10,x20,T1,T2,L1,L2,V1,V2)
   
    
    vel1 = 1/T1 * modulo2pi(x20-L2,x10)
    
    vel2 = 1/T2 * modulo2pi(x10-L1,x20)
    #Velocidades de seguridad.
    
    if vel1 < V1 && vel2 < V2
    
        x1 = taylor1_variable(Float64,20)
        x2 = taylor1_variable(Float64,20)
    
        x1.coeffs[1] = x10
        x1.coeffs[2] = 0
    
        x2.coeffs[1] = x20
        x2.coeffs[2] = 0
        
        for i = 2:20
            x1.coeffs[i] = velocidad_sec(T1,x1,x2,L2).coeffs[i-1]/i
            x2.coeffs[i] = velocidad_sec(T2,x2,x1,L1).coeffs[i-1]/i
        end
    
    
    elseif vel1 < V1 && vel2 > V2
        
        x1 = taylor1_variable(Float64,20)
        x2 = taylor1_variable(Float64,1)
        
        x1.coeffs[1] = x10
        x1.coeffs[2] = 0
        
        x2.coeffs[1] = x20
        
        
        x2.coeffs[2] = V2
        for i = 2:20
            x1.coeffs[i] = velocidad_sec(T1,x1,x2,L2).coeffs[i-1]/i
        end
        
    
    

    
    elseif vel1 > V1 && vel2 < V2
        
        x1 = taylor1_variable(Float64,1)
        x2 = taylor1_variable(Float64,20)
        
        x1.coeffs[1] = x10
        x1.coeffs[2] = V1
        
        x2.coeffs[1] = x20
        x2.coeffs[0] = 0
        
        for i = 2:20
            x2.coeffs[i] = velocidad_sec(T2,x2,x1,L1).coeffs[i-1]/i
        end
        
        
    
    
    elseif vel1 > V1 && vel2 > V2
        x1 = taylor1_variable(Float64,1)
        x2 = taylor1_variable(Float64,1)
        
        x1.coeffs[1] = x10
        x1.coeffs[2] = V1
        
        x2.coeffs[1] = x20
        x2.coeffs[2] = V2
        
    end
    
    return x1, x2
    
end
    
        
    
    
    
    

series (generic function with 1 method)

In [14]:

#t1 y t2 son vectores donde se guardarán los tiempos donde se evalúa la función.
t1 = Float64[];
push!(t1,0);

#x1 y x2 son vectores donde se guardarán los valores de las funciones x1(t) y x2(t)

x1 = Float64[];
push!(x1,x1_0);
x2 = Float64[];
push!(x2,x2_0);




for n in 2:1000
    
    x1Taylor = series(x1_0,x2_0,per1,per2,l1,l2,V1,V2)[1]
    x2Taylor = series(x1_0,x2_0,per1,per2,l1,l2,V1,V2)[2]
    
    h1 = stepsize(x1Taylor,1.0e-5)
    
    h2 = stepsize(x2Taylor,1.0e-5)
    h = min(h1,h2)
    
    
    
    push!(t1,t1[n-1] + h)
    
    x1_0 = evaluate(x1Taylor,h)
    push!(x1,x1_0)
    x2_0 = evaluate(x2Taylor,h)
    push!(x2,x2_0)
    
end
    
    
    
    
    

In [15]:
x1

1000-element Array{Float64,1}:
 0.737743
 0.737748
 0.737753
 0.737758
 0.737763
 0.737768
 0.737773
 0.737778
 0.737783
 0.737788
 0.737793
 0.737798
 0.737803
 ⋮       
 0.742683
 0.742688
 0.742693
 0.742698
 0.742703
 0.742708
 0.742713
 0.742718
 0.742723
 0.742728
 0.742733
 0.742738

In [9]:
h = stepsize(x2,1.0e-20)

LoadError: LoadError: MethodError: `stepsize` has no method matching stepsize(::Array{Float64,1}, ::Float64)
Closest candidates are:
  stepsize{T<:Real}(!Matched::TaylorSeries.Taylor1{T<:Real}, ::Float64)
while loading In[9], in expression starting on line 1

In [16]:
t1

1000-element Array{Float64,1}:
 0.0       
 6.3662e-6 
 1.27324e-5
 1.90986e-5
 2.54648e-5
 3.1831e-5 
 3.81972e-5
 4.45634e-5
 5.09296e-5
 5.72958e-5
 6.3662e-5 
 7.00282e-5
 7.63944e-5
 ⋮         
 0.0062898 
 0.00629617
 0.00630254
 0.0063089 
 0.00631527
 0.00632163
 0.006328  
 0.00633437
 0.00634073
 0.0063471 
 0.00635347
 0.00635983

In [ ]:
function speed(intPos::Array{Float64,1}, carL::Array{Float64,1}, carτ::Array{Float64,1}, carV::Array{Float64,1})
    #Esta función decide la función que debe generar la serie de cada carro.
    
    n = length(intPos)
    F = Function[]
    #F será un arreglo de funciones, una función para cada carro.
    
    for j in 1:n-1
        
        a = (modulo2pi(intPos[j+1]-carlL[j+1],intPos[j]))/carτ[j] #Checa la distancia al coche de enfrente.
        b = carV[j]
        
        if a < b #Compara las velocidades y decide que función aplicar al coche j
            push!(F,fSec)
        else
            push!(F,fmax)
        end
        
    end
    #El coche que está delante del enésimo es el coch 1. Se debe cerrar la cadena.
    
    a = (modulo2pi(intPos[1]-carlL[1],intPos[n]))/carτ[n]
    b = carV[n]
    
    if a < b
        push!(F,fSec)
    else
        push!(F,fmax)
    end
    
    return F
    
end